In [ ]:
# -----------------------------------------------------------
# Query Script: Santa Cruz POI Data from Overture Maps (March 2025 Release)
# Source: Overture Maps Foundation - https://docs.overturemaps.org/
# Data Accessed via: AWS S3 + DuckDB + Jupyter SQL Magic
# Author: Adam Axtopani Gonzales
# Description: Retrieves Points of Interest (POIs) for Santa Cruz, CA,
#              including name, category, and geometry, for mapping and analysis.
# -----------------------------------------------------------


In [4]:
# import our toolkit
import pandas as pd
import geopandas as gpd
import duckdb
from shapely import wkt

In [5]:
# no need to import duckdb_engine, JupySQL will auto-detect driver 
# load (or reload) jupysql Jupyter extension to create SQL cells
%reload_ext sql

In [6]:
# DuckDB in-memory database
%sql duckdb://

Connecting to 'duckdb://'

In [7]:
%sql INSTALL spatial;
%sql INSTALL httpfs;
%sql LOAD spatial;
%sql LOAD httpfs;
%sql SET s3_region='us-west-2'

Running query in 'duckdb://'

Running query in 'duckdb://'

Running query in 'duckdb://'

Running query in 'duckdb://'

Running query in 'duckdb://'

Success


In [27]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

In [ ]:
# This SQL query retrieves a sample of place data (Points of Interest) located in Santa Cruz, California
# from the Overture Maps dataset, which is hosted publicly on AWS S3 in Parquet format. By filtering
# with a geographic bounding box, it ensures that only locations within the Santa Cruz area are returned.
# The query selects each place's unique ID, name, category, and geometry in a readable format. This data
# is useful for mapping, categorization, and analysis of local points of interest, and can support further
# tasks like building custom location-based services, visualizing urban infrastructure, or training
# machine learning models for POI classification.

In [22]:
%%sql santa_cruz_places <<
-- This line uses a Jupyter magic command (%%sql) to run SQL directly in a notebook cell.
-- The result of the query will be saved into the Python variable `santa_cruz_places`.

SELECT
    id,  -- The unique identifier of the place (each POI has a globally unique ID)

    names.primary AS primary_name,  
    -- This selects the main name of the place (e.g., "Joe's Pizza") 
    -- and renames the column to `primary_name` for clarity

    categories.primary AS category,  
    -- This selects the primary category label for the place 
    -- (e.g., "restaurant", "school", or could be NULL)

    ST_AsText(geometry) AS geometry  
    -- This converts the spatial geometry into a readable WKT (Well-Known Text) string 
    -- format so it's easier to view in tables or export to other tools

FROM 
    read_parquet(
        's3://overturemaps-us-west-2/release/2025-03-19.0/theme=places/type=place/*',  
        -- This reads all POI "places" files from Overture Maps' March 2025 release
        filename=true,  -- Includes the file path in the result (helps for debugging if needed)
        hive_partitioning=1  -- Tells DuckDB to interpret directory structure as partition columns
    )

WHERE 
    bbox.xmin >= -122.07 AND bbox.xmax <= -121.97  
    -- Filters for the minimum and maximum longitude values 
    -- (covers a small longitude window around Santa Cruz, CA)

    AND bbox.ymin >= 36.93 AND bbox.ymax <= 37.02  
    -- Filters for the minimum and maximum latitude values 
    -- (covers a small latitude window around Santa Cruz, CA)

LIMIT 100;
-- Limits the result to only 100 rows so the query runs faster and doesn't overload memory



FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [23]:
df = santa_cruz_places

In [24]:
df

,id,primary_name,category,geometry
0,08f28341154f442303cb645263332c9a,Terrace Point,landmark_and_historical_building,POINT (-122.0653984 36.9484061)
1,08f28341154f2c0a0323274055ef3801,Long Marine Laboratory,museum,POINT (-122.0651198 36.94919)
2,08f28341154f2b1e0347a733dbeaeaf6,UCSC Coastal Science Campus,campus_building,POINT (-122.0652825 36.949311)
3,08f28341154f26b60328728031d531bd,UCSC Marine Lab And Seymour Marine Discovery C...,museum,POINT (-122.0649762 36.9492823)
4,08f28341154f243203bcf82dffab43fe,Seymour Marine Discovery Center,zoo,POINT (-122.0649939 36.9493289)
...,...,...,...,...
95,08f2834110198a5a031dd3da41a6bb9e,San Soo Santa Cruz,martial_arts_club,POINT (-122.04937 36.95693)
96,08f2834110198a5a03ca7f1f809d7e44,Santa Cruz Box Company,nursery_and_gardening,POINT (-122.04937 36.95693)
97,08f283411019c65c0390f0cbb1614057,Studio Holladay Weddings,professional_services,POINT (-122.049024 36.9567)
98,08f283411019829c031c2355435bc668,Radical Movement Factory,gym,POINT (-122.049175 36.957094)
